In [1]:
import numpy as np
import pickle

#from train_model import train_step, test_step
from models.decisiontree import DecisionTree, RandForest
from utils.load_data import get_data
from utils.make_dict import train_bow, get_bow

In [2]:
args ={'dataset': 'MNIST',
       'dataroot': './data',
       'model': 'custom_SVM',
       'kernel': 'gaussian',
       'validation': 0.1,
       'dict_size': 200,
       'load_cluster': False
       }

In [3]:
trainX, trainy = get_data(dataset=args['dataset'], train=True, dataroot=args['dataroot'])
if args['dataset'] == 'cifar10':
    trainX = trainX.reshape((-1, 32, 32, 3), order='F')

if args['load_cluster']:
    with open("./cluster.dump", "rb") as f:
        cluster = pickle.load(f)
else:
    cluster = train_bow(trainX, num_dict=args['dict_size'], num_select=10000)
    with open("./cluster.dump", "wb") as f:
        pickle.dump(cluster, f)

trainFeature = get_bow(trainX, cluster, num_dict=args['dict_size'])


In [4]:
_, trainy = get_data(dataset=args['dataset'], train=True, dataroot=args['dataroot'])
with open("./feature.dump", "rb") as f:
    trainFeature = pickle.load(f)

In [5]:
model = RandForest(forest=10, bag_size=10000)
#model = DecisionTree(depth=150)

In [6]:
model.fit(trainFeature, trainy)

100%|██████████| 10/10 [30:34<00:00, 183.49s/it]


In [7]:
y = model.predict(trainFeature)
boolean = (y == trainy)
print(np.sum(boolean) / trainy.shape[0] * 100)

100%|██████████| 10/10 [00:18<00:00,  1.90s/it]

0.0



C:\Users\MLV\anaconda3\envs\vision_project\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [8]:
with open("./cluster.dump", "rb") as f:
    cluster = pickle.load(f)
testX, testy = get_data(dataset=args['dataset'], train=False, dataroot=args['dataroot'])
if args['dataset'] == 'cifar10':
    testX = testX.reshape((-1, 32, 32, 3), order='F')
testFeature = get_bow(testX, cluster, num_dict=args['dict_size'])

In [9]:
pred = model.predict(testFeature)

100%|██████████| 10/10 [00:02<00:00,  3.39it/s]


In [10]:
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [11]:
boolean = (pred == testy)

In [12]:
print(np.sum(boolean) / testy.shape[0] * 100)

11.35
